In [1]:
import numpy as np,os,sys,glob
import matplotlib.pylab as plt
from ioMicro import *

In [2]:
asm = analysis_smFISH(data_folders = [r'Z:\DCBBL1_3_2_2023\MERFISH',r'\\192.168.0.3\bbfishdc10_2\DCBBL1_3_2_2023',r'Z:\DCBBL1_3_2_2023\Controls'],
                 save_folder =r'Z:\DCBBL1_3_2_2023\MERFISH_Analysis',
                 H0folder=  r'Z:\DCBBL1_3_2_2023\Controls\H0*',exclude_H0=False)
#fl_save = r'\\Bbfishdc9\bbfishdc9\DCBBL1_spinal_01_25_2023\psf_Scope2_25_1p5x_cy3_final.pkl'
#dic_psf = pickle.load(open(fl_save,'rb'))
sets = np.unique(['_set'+os.path.basename(fld).split('_set')[-1] for fld in asm.Qfolders if '_set' in os.path.basename(fld)])
print(list(sets))

Found files:1295
Found hybe folders:72
['_set1', '_set2', '_set3', '_set4']


In [18]:
asm.set_fov(0,set_='_set1')
fls = asm.fls_bk_

In [21]:
fl = fls[0]
resc = 5
xs,ys,ims = [],[],[]
for fl in tqdm(fls):
    im,x,y = read_im(fl,return_pos=True)
    ncols,sz,sx,sy = im.shape
    ims.append(np.array(im[-1,sz//2,::resc,::resc],dtype=np.float32))
    xs.append(x)
    ys.append(y)

100%|████████████████████████████████████████████████████████████████████████████████| 340/340 [02:12<00:00,  2.58it/s]


In [30]:
imM,xfov,yfov = compose_mosaic(ims,np.array(ys),-np.array(xs),ims_c=None,um_per_pix=0.108333*resc,rot = 0,return_coords=True)

In [32]:
import napari
viewer = napari.view_image(imM,contrast_limits=[0,5000])
viewer.add_points(np.array([xfov,yfov]).T)

<Points layer 'Points' at 0x23a1b638be0>

In [3]:
ifov=14
asm.set_fov(ifov,set_='_set1')
asm.fov =os.path.basename(asm.fl_bk)

MERFISH_flds = [fld for fld in asm.Qfolders if asm.set_ in os.path.basename(fld) if 'MER' in fld]
ctrl_flds  = [fld for fld in asm.Qfolders if asm.set_ in os.path.basename(fld) if ('ptb' in os.path.basename(fld).lower()) or ('H0' in os.path.basename(fld))]
all_flds = np.array(ctrl_flds+MERFISH_flds)

def get_iH(fld): return int(os.path.basename(fld).split('_')[0][1:])
all_flds = all_flds[np.argsort([get_iH(fld) for fld in all_flds])]

ims = [read_im(fld+os.sep+asm.fov) for fld in all_flds]
ncols,sz,sx,sy = ims[0].shape

In [50]:
ims = [read_im(fld+os.sep+asm.fov) for fld in all_flds]
ncols,sz,sx,sy = ims[0].shape
im_ref = np.array(ims[len(ims)//2][-1],dtype=np.float32)
drifts = [get_txyz(im[-1],im_ref,sz_norm=20, sz=500) for im in tqdm(ims)]
drift_fl = asm.save_folder+os.sep+'drift_'+asm.fov.split('.')[0]+'.pkl'
pickle.dump([drifts,all_flds,asm.fov],open(drift_fl,'wb'))


100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [12:49<00:00, 42.78s/it]


In [53]:
drift_fl = asm.save_folder+os.sep+'drift_'+asm.fov.split('.')[0]+'.pkl'
pickle.dump([drifts,all_flds,asm.fov],open(drift_fl,'wb'))

In [4]:
drift_fl = asm.save_folder+os.sep+'drift_'+asm.fov.split('.')[0]+'.pkl'
drifts,all_flds,fov = pickle.load(open(drift_fl,'rb'))

In [54]:
from dask import array as da
imf = da.concatenate([da.roll(read_im(fld+os.sep+asm.fov),txyz,axis=[1,2,3]).reshape([1,ncols,sz,sx,sy]) 
                      for fld,(txyz,txyzs) in zip(all_flds,drifts)])

In [67]:
fld = all_flds[2]
im_ = read_im(fld+os.sep+asm.fov)
icol=2
im__ = np.array(im_[icol],dtype=np.float32)
im_n = norm_slice(im__,s=30)

In [68]:
for fld in all_flds
Xh = get_local_max(im_n,500,im_raw=im__,dic_psf=None,delta=1,delta_fit=3,dbscan=True,
              return_centers=False,mins=None,sigmaZ=1,sigmaXY=1.5)


In [83]:
ncols

4

In [85]:
#fld = all_flds[0]
for fld in tqdm(all_flds):
    for icol in range(ncols-1):
        im_ = read_im(fld+os.sep+asm.fov)
        #icol=2
        im__ = np.array(im_[icol],dtype=np.float32)
        im_n = norm_slice(im__,s=30)
        Xh = get_local_max(im_n,500,im_raw=im__,dic_psf=None,delta=1,delta_fit=3,dbscan=True,
              return_centers=False,mins=None,sigmaZ=1,sigmaXY=1.5)
        tag = os.path.basename(fld)
        save_fl = asm.save_folder+os.sep+asm.fov.split('.')[0]+'--'+tag+'--col'+str(icol)+'__Xhfits.npy'
        np.savez_compressed(save_fl,Xh=Xh)

100%|█████████████████████████████████████████████████████████████████████████████████| 18/18 [42:33<00:00, 141.85s/it]


In [100]:
#len(Xh)
icol = 0
iH = 1
fld = all_flds[iH]
tag = os.path.basename(fld)
save_fl = asm.save_folder+os.sep+asm.fov.split('.')[0]+'--'+tag+'--col'+str(icol)+'__Xhfits.npy.npz'
Xh = np.load(save_fl)['Xh']

In [145]:
def add_points(Xh,viewer,col='y',szM = 3):

    h = Xh[:,-1]
    hM = np.percentile(h,99.9)
    X = Xh[:,:3]
    size= 1+np.clip(h/hM,0,1)*szM
    text = {
        'string': len(X)*[str(iH)],
        'size': 10,
        'color': col,
    }
    viewer.add_points(
        X,
        size=size,
        face_color='transparent',
        text=None,
        name="",
        edge_color=col,out_of_slice_display=True
    )
def get_Xh(all_flds,fov,drifts,icol = 0,iH = 1):
    fld = all_flds[iH]
    tag = os.path.basename(fld)
    save_fl = asm.save_folder+os.sep+fov.split('.')[0]+'--'+tag+'--col'+str(icol)+'__Xhfits.npy.npz'
    Xh = np.load(save_fl)['Xh']
    tzxy = drifts[iH][0]
    Xh[:,:3]+=tzxy
    return Xh

In [213]:
dec = decoder(analysis_folder = r'\\192.168.0.3\bbfishdc10\DCBBL1_3_2_2023\MERFISH_Analysis')
#dec.get_set_ifov(ifov=ifov,set_=set_,keepH = [1,2,3,4,5,6,7,8],ncols=3)

In [222]:
XH = []
for iH in range(len(all_flds)):
    fld = all_flds[iH]
    if 'MER' in os.path.basename(fld):
        for icol in range(3):
            Xh = get_Xh(all_flds,fov,drifts,icol = icol,iH = iH)#also performs drift correction
            ih = get_iH(fld)
            bit = ih*3+icol
            icolR = np.array([[icol,bit]]*len(Xh))
            XH_ = np.concatenate([Xh,icolR],axis=-1)
            XH.extend(XH_)
XH = np.array(XH)

In [226]:
dec.XH = XH[XH[:,-4]>0.25]

In [233]:
#load_library(dec,lib_fl = r'codebook_DCBB250.csv',nblanks=-1)
load_library(dec,lib_fl = r'Z:\DCBBL1_3_2_2023\MERFISH_Analysis\codebook_0_New_DCBB-300_MERFISH_encoding_2_21_2023.csv',nblanks=-1)

In [242]:
get_inters(dec,dinstance_th=2,enforce_color=True)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:41<00:00, 33.91s/it]


In [243]:
dec.pick_best_brightness(nUR_cutoff = 3,resample = 50000)

100%|█████████████████████████████████████████████████████████████████████████| 50038/50038 [00:03<00:00, 16577.20it/s]


In [247]:
#len(dec.res)
dec.res[0]

array([0], dtype=int64)

[-1, 3696, -1, 9887656]

In [250]:
self = dec
#self.get_brightness_distance_distribution()
res =self.res
XH = self.XH
codes = self.codes__

RS = XH[:,-1].astype(int)
HS = XH[:,-3]
colS = XH[:,-2].astype(int)
colSu = np.unique(colS)
meds_col = np.array([np.median(HS[colS==cu]) for cu in colSu])

self.HN = HS/meds_col[colS]

In [264]:
from tqdm import tqdm
res_pruned = []
nmin = 3
for r in tqdm(res):
    if len(r)>nmin:
        R_ = RS[r]
        Ru = np.unique(R_)
        if len(Ru)>nmin:
            hn = self.HN[r]
            X = self.XH[r,:3]
            dn = np.linalg.norm(X-np.mean(X,axis=0),axis=-1)
            #sc_dist = self.get_score_distance(dn)
            #sc_br = self.get_score_brightness(hn)
            #scores = sc_dist*sc_br
            #[R_==R for R in Ru]




  0%|                                                                                     | 0/15411481 [00:00<?, ?it/s]


  0%|                                                                     | 14513/15411481 [00:00<01:46, 144988.41it/s]


  0%|▏                                                                    | 29012/15411481 [00:00<02:26, 105180.07it/s]


  0%|▏                                                                     | 40171/15411481 [00:00<02:41, 95327.34it/s]


  0%|▏                                                                     | 50037/15411481 [00:00<02:53, 88716.03it/s]


  0%|▎                                                                     | 59081/15411481 [00:00<02:56, 87125.64it/s]


  0%|▎                                                                     | 67881/15411481 [00:00<03:12, 79574.52it/s]


  0%|▎                                                                     | 75933/15411481 [00:00<03:17, 77818.74it/s]


  1%|▍               

KeyboardInterrupt: 

In [287]:
#### unfold res which is a list of list with clusters of loc.
import time
start = time.time()
nmin=3
res = [r for r in res if len(r)>nmin]
rlens = [len(r) for r in res]
edges = np.cumsum([0]+rlens)
res_unfolder = [r_ for r in res for r_ in r]
res0 = [r[0] for r in res for r_ in r]
ires = np.array([ir for ir,r in enumerate(res) for r_ in r])
print(time.time()-start)

### compute the scores
import time
start = time.time()
hn = self.HN[res_unfolder]
X = self.XH[:,:3][res_unfolder]
X0 = self.XH[:,:3][res0]
dn = np.linalg.norm(X-X0,axis=-1)
sc_dist = self.get_score_distance(dn)
sc_br = self.get_score_brightness(hn)
scores = sc_dist*sc_br
print(time.time()-start)

25.901713848114014


In [ ]:
Rs = self.XH[:,-1]

In [315]:
uRs = ires*(np.max(Rs)+1)+Rs[res_unfolder]

In [317]:
arg_scores = np.argsort(scores)[::-1]
uRs = uRs[arg_scores]

In [328]:
uvals,inv = np.unique(uRs,return_index=True)

In [343]:
Rs = self.XH[:,-1]
bits = (Rs-3).astype(int)##modify
bits_unfold = bits[res_unfolder]
nbits = len(np.unique(bits))
scores_bits = np.zeros([len(res),nbits])

arg_scores = np.argsort(scores)

In [350]:
arg_scores = np.argsort(scores)

In [351]:
scores_bits[ires[arg_scores],bits_unfold[arg_scores]]=scores[arg_scores]
icodes = np.argmax(np.dot(scores_bits[:],self.codes_01.T),axis=-1)

In [359]:
scores_bits

(8763001, 48)

In [371]:
icodes = np.argmax(np.dot(scores_bits[:],self.codes_01.T),axis=-1)

array([320, 315, 320, ..., 269, 299, 299], dtype=int64)

In [ ]:
tree = KDTree(self.codes_01)
_,icodes = tree.query(scores_bits[:],p=1)

In [357]:
r = res[6]
r

array([     307,  1059105,  7161879,  8089567,  9877224, 13851737],
      dtype=int64)

In [361]:
self.codes_01.shape

(575, 48)

In [334]:
nRs = len(np.unique(Rs))

array([ 3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14., 15.,
       16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26., 27., 28.,
       29., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39., 40., 41.,
       42., 43., 44., 45., 46., 47., 48., 49., 50.])

In [333]:
len(inv)

48700373

In [358]:
test=np.zeros(9)
test[[2,3,4,2,2]]=[1,1,1,1,0.5]
test

array([0. , 0. , 0.5, 1. , 1. , 0. , 0. , 0. , 0. ])

In [319]:
np.unique(uRs,retur)

array([2.65845664e+08, 2.04208140e+07, 1.39030161e+08, ...,
       2.66855479e+08, 2.43936454e+08, 2.37273811e+08])

In [297]:
0.99*scores+ires*(np.max(Rs)+1)

87.53037571907043


In [316]:
uRs

array([3.00000000e+00, 6.00000000e+00, 2.40000000e+01, ...,
       4.46913044e+08, 4.46913044e+08, 4.46913050e+08])

In [ ]:
res_refolded = [res_unfolder[edges[ie]:edges[ie+1]] for ie in range(len(edges)-1)]

In [309]:
ires*100

array([        0,         0,         0, ..., 876300000, 876300000,
       876300000])

In [313]:

np.min(Rs)

3.0

In [307]:
#res_refolded
np.argsort(scores+ires)

array([       1,        3,        2, ..., 55242467, 55242465, 55242464],
      dtype=int64)

In [303]:
scores

array([7.94826997e-02, 2.01109707e-03, 3.06271622e-02, ...,
       1.16140412e-02, 3.35209731e-06, 1.10825370e-03])

In [286]:
res_refolded[30000]==res[30000]

array([ True,  True,  True,  True,  True,  True,  True])

In [277]:
res_refolded = []

68718621

In [ ]:
02:24

In [ ]:
def pick_best_score(self,nUR_cutoff = 3,resample=1):
    """Pick the best code for each molecular cluster based on the fisher statistics 
    for the distance and brightness distribution"""
    self.get_brightness_distance_distribution()
    res =self.res
    XH = self.XH
    codes = self.codes__

    RS = XH[:,-1].astype(int)
    HS = XH[:,-3]
    colS = XH[:,-2].astype(int)
    colSu = np.unique(colS)
    meds_col = np.array([np.median(HS[colS==cu]) for cu in colSu])

    self.HN = HS/meds_col[colS]

    ncodes = len(codes)

    bucket = np.zeros(ncodes)
    nbits_per_code = np.array([len(cd) for cd in codes])

    icodes = []
    res_pruned = []
    scores_pruned = []
    for r in tqdm(res[::resample]):
        hn = self.HN[r]
        X = XH[r,:3]
        dn = np.linalg.norm(X-np.mean(X,axis=0),axis=-1)
        sc_dist = self.get_score_distance(dn)
        sc_br = self.get_score_brightness(hn)
        scores = sc_dist*sc_br
        dic_sc = {r:sc for r,sc in zip(r,scores)}
        isort = np.argsort(scores)
        r = np.array(r)[isort]
        #scores = scores[isort]
        R = RS[r]
        dic_u = {R_:r_ for r_,R_ in zip(r,R)}
        if len(dic_u)>=nUR_cutoff:
            bucket_ = np.zeros(ncodes)
            for R_ in dic_u:
                if R_ in self.dic_bit_to_code:
                    icds = self.dic_bit_to_code[R_]
                    bucket_[icds]+=dic_sc[dic_u[R_]]
            bucket_/=nbits_per_code
            best_code = np.argmax(bucket_)
            icodes.append(best_code)
            rf = [dic_u.get(R_,-1) for R_ in codes[best_code]]
            res_pruned.append(rf)
            scores_pruned.append([dic_sc.get(r_,-1000) for r_ in rf])
    self.res_pruned = np.array(res_pruned)
    self.icodes = np.array(icodes)
    self.scores_pruned = np.array(scores_pruned)
    X1f,X2f = self.dic_pair.get(0,[[[0,0,0]],[[0,0,0]]])
    driftf = np.mean(np.array(X1f)-X2f,axis=0)
    X = self.XH[:,:3]-driftf
    res_ = np.array(self.res_pruned)
    keep = (res_>=0)[...,np.newaxis]
    self.Xcms = np.sum(X[res_]*keep,axis=1)/np.sum(keep,axis=1)

In [248]:
??dec.pick_best_score

In [244]:
dec.pick_best_score(nUR_cutoff = 3)

  7%|████▊                                                                | 1063358/15411481 [02:52<38:47, 6165.34it/s]


KeyboardInterrupt: 

In [241]:
def get_inters(self,dinstance_th=2,enforce_color=False):
    """Get an initial intersection of points and save in self.res"""
    res =[]
    if enforce_color:
        icols = self.XH[:,-2]
        XH = self.XH
        for icol in tqdm(np.unique(icols)):
            inds = np.where(icols==icol)[0]
            Xs = XH[inds,:3]
            Ts = cKDTree(Xs)
            res_ = Ts.query_ball_tree(Ts,dinstance_th)
            res += [inds[r] for r in res_]
    else:
        XH = self.XH
        Xs = XH[:,:3]
        Ts = cKDTree(Xs)
        res = Ts.query_ball_tree(Ts,dinstance_th)
    self.res = res

In [237]:
icols = dec.XH[:,-2]
icols[]
np.unique(dec.XH[:,-2],return_counts=True)

(array([0., 1., 2.]), array([5295698, 6908946, 3206837], dtype=int64))

In [230]:
import ioMicro
ioMicro.__file__

'C:\\Users\\BintuLab\\Scope4AnalysisScripts\\MERFISH_Spot_Analysis\\Analysis_1500gns_MERFISH\\ioMicro.py'

In [228]:
??dec.load_library

In [130]:
#dec.get_XH()
print("Correcting for distortion acrossbits")
#dec.apply_distortion_correction()
dec.load_library(lib_fl=master_analysis_folder+os.sep+'codebook_DCBB250.csv')
dec.get_inters(dinstance_th=3)
dec.pick_best_brightness(nUR_cutoff = 3,resample = 10000)
dec.pick_best_score(nUR_cutoff = 3)
np.savez_compressed(dec.save_file_dec,res_pruned=dec.res_pruned,icodes=dec.icodes,scores_pruned = dec.scores_pruned,Xcms=dec.Xcms)

In [128]:
drift_fl = asm.save_folder+os.sep+'drift_'+asm.fov.split('.')[0]+'.pkl'
drifts,all_flds,fov = pickle.load(open(drift_fl,'rb'))

In [146]:

import napari
viewer = napari.Viewer()


Xh = get_Xh(all_flds,fov,drifts,icol = 2,iH = 0)
add_points(Xh,viewer,col='w')
Xh = get_Xh(all_flds,fov,drifts,icol = 2,iH = 1)
add_points(Xh,viewer,col='y')
Xh = get_Xh(all_flds,fov,drifts,icol = 2,iH = 2)
add_points(Xh,viewer,col='r')
Xh = get_Xh(all_flds,fov,drifts,icol = 2,iH = 3)
add_points(Xh,viewer,col='g')
Xh = get_Xh(all_flds,fov,drifts,icol = 2,iH = 6)
add_points(Xh,viewer,col='b')
Xh = get_Xh(all_flds,fov,drifts,icol = 2,iH = 9)
add_points(Xh,viewer,col='c')

In [160]:
MER = [2, 3, 6, 9]
icol=2
XH = []
for iH in MER:
    Xh = get_Xh(all_flds,fov,drifts,icol = icol,iH = iH)
    Rcol = np.array([[icol,iH]]*len(Xh))
    XH.extend(np.concatenate([Xh,Rcol],axis=-1))
XH = np.array(XH)

In [165]:
from scipy.spatial import KDTree
dinstance_th=2
Xs = XH[:,:3]
Ts = KDTree(Xs)
res = Ts.query_ball_tree(Ts,dinstance_th)
res = [r for r in res if len(r)>=4]

In [166]:
Rs = XH[:,-1]
res = [r for r in res if len(np.unique(Rs[r]))>=4]
len(res)

83335

In [176]:
hs = XH[:,-3]
res_ = []
for r in res:
    if len(r)>len(MER):
        r_ = np.array(r)
        rs_ = [np.array(r_)[Rs[r_]==R] for R in MER]
        br = [r_[np.argmax(hs[r_])]for r_ in rs_]
    else:
        br=r
    res_.append(br)
Xdec = np.array([np.mean(Xs[r],axis=0) for r in res_])
hdec = np.array([np.mean(hs[r],axis=0) for r in res_])

In [203]:
_,inv = np.unique(['_'.join([str(z),str(x),str(y)])for z,x,y in Xdec.astype(int)],return_index=True)
Xdec_ = Xdec[inv]
hdec_ = hdec[inv]

In [208]:
import napari
viewer = napari.Viewer()
Xh = get_Xh(all_flds,fov,drifts,icol = 2,iH = 0)
add_points(Xh,viewer,col='w')
Xh = get_Xh(all_flds,fov,drifts,icol = 2,iH = 1)
add_points(Xh,viewer,col='y')
Xh_ = np.array([np.mean(XH[r,:-3],axis=0) for r in res_])[inv]
add_points(Xh_,viewer,col='r')

array([  520.70495605,   521.00799561,   521.63006592, ...,
       12920.2142334 , 13459.79467773, 15659.33056641])

In [ ]:
XH = self.XH
Xs = XH[:,:3]
Ts = cKDTree(Xs)
res = Ts.query_ball_tree(Ts,dinstance_th)
self.res = res

In [94]:
features = {
    'likelihood': [21.23423, 51.2315, 100],
    'class': ['hand', 'face', 'camera'],
}
edge_color_cycle = ['blue', 'magenta', 'green']

text = {
    'string': '{class}: {likelihood:0.1f}%',
    'anchor': 'upper_left',
    'translation': [-5, 0],
    'size': 8,
    'color': 'green',
}
shapes_layer = viewer.add_points(
    np.array([[0,1],[1,2],[2,2]]),
    features=features,
    edge_color='class',
    edge_color_cycle=edge_color_cycle,
    face_color='transparent',
    text=text,
    name='shapes',
)

In [ ]:
import napari
viewer = napari.Viewer()

In [13]:
drifts[iQ][0]

array([  0,  87, 115])

In [22]:
iQ = 2
from dask import array as da
im_n = norm_slice(im__,s=50)
viewer.add_image(np.roll(im_n,drifts[iQ][0],axis=[0,1,2]))
iQ = 1
viewer.add_image(da.roll(read_im(all_flds[iQ]+os.sep+asm.fov),drifts[iQ][0],axis=[1,2,3])[icol])

<Image layer 'Image [1]' at 0x2f129807f40>

In [29]:
imT = np.array(read_im(all_flds[iQ]+os.sep+asm.fov)[icol],dtype=np.float32)

In [33]:
napari.view_image()

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 249.5, 249.5), zoom=1.0146, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(1.0, 1.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=3, ndisplay=2, last_used=0, range=((0.0, 37.0, 1.0), (0.0, 500.0, 1.0), (0.0, 500.0, 1.0)), current_step=(18, 250, 250), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'Image' at 0x2f1ba3516d0>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10, unit=None), text_overlay=TextOverlay(visible=False, color=(0.5, 0.5, 0.5, 1.0), font_size=10, position=<TextOverlayPosition.TOP_LEFT: 'top_left'>, text=''), overlays=Overlays(interaction_box=InteractionBox(points=None, show=False, show_handle=False, show_vertices=False, selecti

In [37]:
imT_ = norm_slice(imT[:,500:1000,500:1000],s=30)

In [38]:
napari.view_image(imT_)

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 249.5, 249.5), zoom=2.3313, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(1.0, 1.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=3, ndisplay=2, last_used=0, range=((0.0, 37.0, 1.0), (0.0, 500.0, 1.0), (0.0, 500.0, 1.0)), current_step=(18, 250, 250), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'imT_' at 0x2f3504f7ca0>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10, unit=None), text_overlay=TextOverlay(visible=False, color=(0.5, 0.5, 0.5, 1.0), font_size=10, position=<TextOverlayPosition.TOP_LEFT: 'top_left'>, text=''), overlays=Overlays(interaction_box=InteractionBox(points=None, show=False, show_handle=False, show_vertices=False, selectio

In [66]:
Hs = []
paramas =[]
for sigmaZ in tqdm(np.arange(0.5,2,0.2)):
    for sigmaXY in np.arange(0.5,2.5,0.25):
        Xh = get_local_max(imT_,1000,im_raw=None,dic_psf=None,delta=1,delta_fit=3,dbscan=True,return_centers=False,mins=None,sigmaZ=sigmaZ,sigmaXY=sigmaXY)
        Hs.append(np.median(Xh[:,-2]))
        paramas.append([sigmaZ,sigmaXY])

  0%|                                                                                            | 0/8 [00:00<?, ?it/s]C:\Users\BintuLab\AppData\Local\Temp\ipykernel_15444\1722703158.py:58: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  im_centers_ = np.array(im_centers)
100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:06<00:00,  1.16it/s]


In [65]:
Hs

[0.35682440387054193,
 0.4777019465771151,
 0.5266061579705601,
 0.531471835279339,
 0.5149107759492777,
 0.4920134092310091,
 0.4530320861173032,
 0.42166541489499554,
 0.4019617585244928,
 0.5421978778895774,
 0.5993361593781625,
 0.6207828520615402,
 0.6187324329398474,
 0.6001368479274907,
 0.5588916037916883,
 0.522434407461402,
 0.4035767223099017,
 0.5611276020341365,
 0.631198128111368,
 0.6512938283634699,
 0.6583785179922257,
 0.6525826688637297,
 0.6203418113355859,
 0.586122508057862,
 0.39853047387558627,
 0.5597305330054305,
 0.6351976564270234,
 0.6667777463166176,
 0.6771028128368013,
 0.6743728542225171,
 0.654116712882578,
 0.6213012367988969,
 0.38569125569233176,
 0.5426780053182625,
 0.6186632888267203,
 0.664809594503716,
 0.6761181601551458,
 0.6813549604450081,
 0.6727891063233535,
 0.6454782419581843,
 0.36727699586335627,
 0.5228672388093061,
 0.5915414547420109,
 0.6389362311564662,
 0.665742850580322,
 0.6753738338763697,
 0.6770136347851483,
 0.662599095729

In [34]:
def get_local_max(im_dif,th_fit,im_raw=None,dic_psf=None,delta=1,delta_fit=3,dbscan=True,return_centers=False,mins=None,
                  sigmaZ=1,sigmaXY=1.5):
    """Given a 3D image <im_dif> as numpy array, get the local maxima in cube -<delta>_to_<delta> in 3D.
    Optional a dbscan can be used to couple connected pixels with the same local maximum. 
    (This is important if saturating the camera values.)
    Returns: Xh - a list of z,x,y and brightness of the local maxima
    """
    
    z,x,y = np.where(im_dif>th_fit)
    zmax,xmax,ymax = im_dif.shape
    in_im = im_dif[z,x,y]
    keep = np.ones(len(x))>0
    for d1 in range(-delta,delta+1):
        for d2 in range(-delta,delta+1):
            for d3 in range(-delta,delta+1):
                keep &= (in_im>=im_dif[(z+d1)%zmax,(x+d2)%xmax,(y+d3)%ymax])
    z,x,y = z[keep],x[keep],y[keep]
    h = in_im[keep]
    Xh = np.array([z,x,y,h]).T
    if dbscan and len(Xh)>0:
        from scipy import ndimage
        im_keep = np.zeros(im_dif.shape,dtype=bool)
        im_keep[z,x,y]=True
        lbl, nlbl = ndimage.label(im_keep,structure=np.ones([3]*3))
        l=lbl[z,x,y]#labels after reconnection
        ul = np.arange(1,nlbl+1)
        il = np.argsort(l)
        l=l[il]
        z,x,y,h = z[il],x[il],y[il],h[il]
        inds = np.searchsorted(l,ul)
        Xh = np.array([z,x,y,h]).T
        Xh_ = []
        for i_ in range(len(inds)):
            j_=inds[i_+1] if i_<len(inds)-1 else len(Xh)
            Xh_.append(np.mean(Xh[inds[i_]:j_],0))
        Xh=np.array(Xh_)
        z,x,y,h = Xh.T
    im_centers=[]
    if delta_fit!=0 and len(Xh)>0:
        z,x,y,h = Xh.T
        z,x,y = z.astype(int),x.astype(int),y.astype(int)
        im_centers = [[],[],[],[],[]]
        Xft = []
        
        for d1 in range(-delta_fit,delta_fit+1):
            for d2 in range(-delta_fit,delta_fit+1):
                for d3 in range(-delta_fit,delta_fit+1):
                    if (d1*d1+d2*d2+d3*d3)<=(delta_fit*delta_fit):
                        im_centers[0].append((z+d1))
                        im_centers[1].append((x+d2))
                        im_centers[2].append((y+d3))
                        im_centers[3].append(im_dif[(z+d1)%zmax,(x+d2)%xmax,(y+d3)%ymax])
                        if im_raw is not None:
                            im_centers[4].append(im_raw[(z+d1)%zmax,(x+d2)%xmax,(y+d3)%ymax])
                        Xft.append([d1,d2,d3])
              
        Xft = np.array(Xft)
        im_centers_ = np.array(im_centers)
        bk = np.min(im_centers_[3],axis=0)
        im_centers_[3] -= bk
        a = np.sum(im_centers_[3],axis=0)
        habs = np.zeros_like(bk)
        if im_raw is not None:
            habs = im_raw[z%zmax,x%xmax,y%ymax]
          
        if dic_psf is not None:
            keys = list(dic_psf.keys())
            ### calculate spacing
            im0 = dic_psf[keys[0]]
            space = np.sort(np.diff(keys,axis=0).ravel())
            space = space[space!=0][0]
            ### convert to reduced space
            zi,xi,yi = (z/space).astype(int),(x/space).astype(int),(y/space).astype(int)

            keys_ =  np.array(keys)
            sz_ = list(np.max(keys_//space,axis=0)+1)

            ind_ = tuple(Xft.T+np.array(im0.shape)[:,np.newaxis]//2-1)

            im_psf = np.zeros(sz_+[len(ind_[0])])
            for key in keys_:
                coord = tuple((key/space).astype(int))
                im__ = dic_psf[tuple(key)][ind_]
                im_psf[coord]=(im__-np.mean(im__))/np.std(im__)
            im_psf_ = im_psf[zi,xi,yi]
            im_centers__ = im_centers_[3].T.copy()
            im_centers__ = (im_centers__-np.mean(im_centers__,axis=-1)[:,np.newaxis])/np.std(im_centers__,axis=-1)[:,np.newaxis]
            hn = np.mean(im_centers__*im_psf_,axis=-1)
        else:

            #im_sm = im_[tuple([slice(x_-sz,x_+sz+1) for x_ in Xc])]
            sz = delta_fit
            Xft = (np.indices([2*sz+1]*3)-sz).reshape([3,-1]).T
            Xft = Xft[np.linalg.norm(Xft,axis=1)<=sz]
            
            sigma = np.array([sigmaZ,sigmaXY,sigmaXY])[np.newaxis]
            Xft_ = Xft/sigma
            norm_G = np.exp(-np.sum(Xft_*Xft_,axis=-1)/2.)
            norm_G=(norm_G-np.mean(norm_G))/np.std(norm_G)
            im_centers__ = im_centers_[3].T.copy()
            im_centers__ = (im_centers__-np.mean(im_centers__,axis=-1)[:,np.newaxis])/np.std(im_centers__,axis=-1)[:,np.newaxis]
            hn = np.mean(im_centers__*norm_G,axis=-1)
        
        zc = np.sum(im_centers_[0]*im_centers_[3],axis=0)/np.sum(im_centers_[3],axis=0)
        xc = np.sum(im_centers_[1]*im_centers_[3],axis=0)/np.sum(im_centers_[3],axis=0)
        yc = np.sum(im_centers_[2]*im_centers_[3],axis=0)/np.sum(im_centers_[3],axis=0)
        Xh = np.array([zc,xc,yc,bk,a,habs,hn,h]).T
    if return_centers:
        return Xh,np.array(im_centers)
    return Xh

In [26]:
import ioMicro
!"{}"

The system cannot execute the specified program.


In [27]:
print(ioMicro.__file__)

C:\Users\BintuLab\Scope4AnalysisScripts\MERFISH_Spot_Analysis\Analysis_1500gns_MERFISH\ioMicro.py


In [67]:
import napari
napari.view_image(im_n)

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 1499.5, 1499.5), zoom=0.16909999999999997, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(1.0, 1.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=3, ndisplay=2, last_used=0, range=((0.0, 37.0, 1.0), (0.0, 3000.0, 1.0), (0.0, 3000.0, 1.0)), current_step=(18, 1500, 1500), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'im_n' at 0x23986840670>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10, unit=None), text_overlay=TextOverlay(visible=False, color=(0.5, 0.5, 0.5, 1.0), font_size=10, position=<TextOverlayPosition.TOP_LEFT: 'top_left'>, text=''), overlays=Overlays(interaction_box=InteractionBox(points=None, show=False, show_handle=False, show_verti

In [86]:
import pandas
codebook = pandas.read_csv(r'Z:\DCBB_1500gnLib_01_23_2023\MERFISH\New_DCBB-300_MERFISH_encoding_2_21_2023.csv')
import numpy as np
code_gene = codebook[codebook['name']=='Ptbp1'].iloc[:,2:]
code = np.where(code_gene.iloc[0])[0]
icol =code[0]%3 
hybes =code//3
hybes+2,icol

(array([2, 3, 6, 9], dtype=int64), 2)

In [89]:
import pandas
codebook = pandas.read_csv(r'Z:\DCBB_1500gnLib_01_23_2023\MERFISH\New_DCBB-300_MERFISH_encoding_2_21_2023.csv')
import numpy as np
code_gene = codebook[codebook['name']=='Igfbpl1'].iloc[:,2:]
code = np.where(code_gene.iloc[0])[0]
icol =code[0]%3 
hybes =code//3 
hybes+2,icol

(array([2, 5, 7, 8], dtype=int64), 0)

In [20]:
set_flds = [fld for fld in asm.Qfolders if asm.set_ in os.path.basename(fld)]
iPTB = [ifld for ifld,fld in enumerate(set_flds) if 'PTB' in os.path.basename(fld)][0]
iPTB

21

In [21]:
asm.set_hybe(iPTB)

completed = asm.check_finished_file()
if not completed:
    asm.get_background(force=False)
    asm.get_signal()
    asm.compute_drift(sz=300)
    #asm.get_aligned_ims()
    #imf = asm.im_sig__/asm.im_bk__
    #resc=5
    #asm.im_ration = np.array([norm_slice(im_,s=30)/np.median(np.abs(im_[::resc,::resc,::resc])) for im_ in asm.im_sig_])
    #asm.im_ration = np.array([norm_slice(im_,s=30)/np.median(np.abs(im_[::resc,::resc,::resc])) for im_ in asm.im_sig_])

    asm.dic_th = {0:1.25,1:0.5,2:1.25}
    asm.get_Xh(th = 3)
    asm.save_fits(plt_val=False)

### define H0 and load background
Background file: Z:\DCBB_1500gnLib_01_23_2023\Controls\H0_set1\Conv_zscan__250.zarr
### load signal
Signal file: Z:\DCBB_1500gnLib_01_23_2023\Controls\H1_PTB_set1\Conv_zscan__250.zarr
Found drift:[ 7 -2  5]


NameError: name 'dic_psf' is not defined

In [26]:
import napari
napari.view_image(asm.im_sig_)

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 1499.5, 1499.5), zoom=0.38158333333333333, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(1.0, 1.0, 0.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=4, ndisplay=2, last_used=0, range=((0.0, 4.0, 1.0), (0.0, 37.0, 1.0), (0.0, 3000.0, 1.0), (0.0, 3000.0, 1.0)), current_step=(2, 18, 1500, 1500), order=(0, 1, 2, 3), axis_labels=('0', '1', '2', '3')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'Image' at 0x13cf6fd7ac0>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10, unit=None), text_overlay=TextOverlay(visible=False, color=(0.5, 0.5, 0.5, 1.0), font_size=10, position=<TextOverlayPosition.TOP_LEFT: 'top_left'>, text=''), overlays=Overlays(interaction_box=InteractionBox(points=None, show=Fa

In [22]:
?asm.get_Xh

In [16]:
#asm.Qfolders = [asm.Qfolders]


['Z:\\DCBB_1500gnLib_01_23_2023\\MERFISH\\H10_MERRep_set1',
 'Z:\\DCBB_1500gnLib_01_23_2023\\MERFISH\\H10_MERRep_set2',
 'Z:\\DCBB_1500gnLib_01_23_2023\\MERFISH\\H11_MERRep_set1',
 'Z:\\DCBB_1500gnLib_01_23_2023\\MERFISH\\H11_MERRep_set2',
 'Z:\\DCBB_1500gnLib_01_23_2023\\MERFISH\\H12_MERRep_set1',
 'Z:\\DCBB_1500gnLib_01_23_2023\\MERFISH\\H12_MERRep_set2',
 'Z:\\DCBB_1500gnLib_01_23_2023\\MERFISH\\H13_MERRep_set1',
 'Z:\\DCBB_1500gnLib_01_23_2023\\MERFISH\\H13_MERRep_set2',
 'Z:\\DCBB_1500gnLib_01_23_2023\\MERFISH\\H14_MER_set1',
 'Z:\\DCBB_1500gnLib_01_23_2023\\MERFISH\\H14_MER_set2',
 'Z:\\DCBB_1500gnLib_01_23_2023\\MERFISH\\H15_MER_set1',
 'Z:\\DCBB_1500gnLib_01_23_2023\\MERFISH\\H15_MER_set2',
 'Z:\\DCBB_1500gnLib_01_23_2023\\MERFISH\\H16_MER_set1',
 'Z:\\DCBB_1500gnLib_01_23_2023\\MERFISH\\H16_MER_set2',
 'Z:\\DCBB_1500gnLib_01_23_2023\\MERFISH\\H1_MERRep_set1',
 'Z:\\DCBB_1500gnLib_01_23_2023\\MERFISH\\H1_MERRep_set2',
 'Z:\\DCBB_1500gnLib_01_23_2023\\MERFISH\\H2_MER_set1',
 'Z:

In [8]:
?asm.set_fov

In [ ]:
def process_fov(ifov):
    asm.set_fov(ifov)
    final_segmentation(asm.fl_bk,
                        analysis_folder=asm.save_folder,
                        plt_val=True,
                        rescz = 3,trimz=2, resc=4,p99=850)
    if True:
        for iQ in range(nQs):

            asm.set_hybe(iQ)

            completed = asm.check_finished_file()
            if not completed:
                asm.get_background(force=False)
                asm.get_signal()
                asm.compute_drift(sz=300)
                #asm.get_aligned_ims()
                #imf = asm.im_sig__/asm.im_bk__
                resc=5
                #asm.im_ration = np.array([norm_slice(im_,s=30)/np.median(np.abs(im_[::resc,::resc,::resc])) for im_ in asm.im_sig_])
                #asm.im_ration = np.array([norm_slice(im_,s=30)/np.median(np.abs(im_[::resc,::resc,::resc])) for im_ in asm.im_sig_])

                asm.dic_th = {0:1.25,1:0.5,2:1.25}
                asm.get_Xh(th = 3,dic_psf=dic_psf)
                asm.save_fits(plt_val=False)